In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.level = logging.INFO

from os import path
from corputil import FileCorpus
from gensim.models import Doc2Vec
from gensim.models.doc2vec import LabeledSentence